# Setup: Connectivity

In [1]:

from katana import remote
   #
from IPython.display import display
from ipywidgets import Layout, Textarea, VBox
   #
from libraries.SupportFor34 import f_init, f_enrich
    
    
my_client, my_graph = f_init()


print("My Client: %s" % (my_client))
print("My Graph:  %s" % (my_graph ))
   #
print()


My Client: <katana_enterprise.remote.sync_wrappers.Client object at 0x7f45545e2a30>
My Graph:  <_Graph my_graph, DzYVK5XAQEaYDuU3z9kAqbzEK7sth9Pt3Pc3NVgVY53j, 4>



#  Step 01: Enter a new Medical Description

In [2]:

#  Create a text entry area
#

l_layout   = Layout(flex = "0 1 auto", height = "100px", min_height = "100px", width = "auto")
l_text01   = Textarea(value="Odd mole on my ear", layout = l_layout, description = "Enter Text: ")
   #
l_vbox     = VBox([l_text01], layout = Layout(flex = "1 1 auto", width = "auto"))
display(l_text01)


Textarea(value='Odd mole on my ear', description='Enter Text: ', layout=Layout(flex='0 1 auto', height='100px'…

In [3]:

#  Enrich/decode text entry from above
#

l_return, l_pvid = f_enrich(l_text01.value.lower(), my_graph)

print("Unique Patient Visit Id: %s" % (l_pvid))
print("")
print("Medical Description encoding:")
print("")
display(l_return)


          0/? [?op/s]

Unique Patient Visit Id: PV-2799

Medical Description encoding:



{'entityMentions': [{'mentionId': '1',
   'type': 'PROBLEM',
   'text': {'content': 'mole', 'beginOffset': 4},
   'linkedEntities': [{'entityId': 'UMLS/C0027960'},
    {'entityId': 'UMLS/C0027962'}],
   'temporalAssessment': {'value': 'UPCOMING',
    'confidence': 0.6084825992584229},
   'certaintyAssessment': {'value': 'LIKELY',
    'confidence': 0.057481713593006134},
   'subject': {'value': 'PATIENT', 'confidence': 0.9589220881462097},
   'confidence': 0.5596652626991272},
  {'mentionId': '2',
   'type': 'ANATOMICAL_STRUCTURE',
   'text': {'content': 'ear', 'beginOffset': 15},
   'linkedEntities': [{'entityId': 'UMLS/C0013443'}],
   'confidence': 0.9025713205337524}],
 'entities': [{'entityId': 'UMLS/C0013443',
   'preferredTerm': 'Ear structure',
   'vocabularyCodes': ['FMA/52780',
    'FMA/77739',
    'LNC/LA21929-7',
    'LNC/LA22163-2',
    'LNC/LP7188-8',
    'LNC/MTHU001407',
    'MSH/D004423',
    'MTH/NOCODE',
    'NCI/C12394',
    'OMIM/MTHU000052']},
  {'entityId': 'UMLS/C

# Step 02:  Examine the Graph

In [ ]:

#  Produce a visual graph

from katana_visualization_widget import GraphVisOptions, NodeVisOption, EdgeVisOption, ANY

l_options = GraphVisOptions(
   node_options = [
      NodeVisOption("PatientVisit",     label="short_text"       ),
      NodeVisOption("UmlsEntity",       label="id"               ),
      NodeVisOption("UmlsVocabulary",   label="id"               ),
   ],
   #  edge_options = [
   #     EdgeVisOption(["VISIT_CONTAINS"], label="start_id"),
   #     EdgeVisOption(["ALSO_CODED_AS" ], label="start_id"),
   #  ]
   )
    
    
l_query  = """

   MATCH (pv: PatientVisit) - [vc: VISIT_CONTAINS] -> (ue: UmlsEntity) - [aca: ALSO_CODED_AS] -> (uv: UmlsVocabulary)
   WHERE
      (pv.id = '{0}')
   RETURN pv, vc, ue, aca, uv
   
   """.format(l_pvid)

l_result = my_graph.query_unpaginated(l_query, contextualize=True)
    
l_result.view(graph_vis_options = l_options)


In [ ]:

#  Checking just nodes
#

l_query  = """

   MATCH (n)
   WHERE NOT n.id STARTS WITH 'XX-'                   //  A value we use as a placeholder
   RETURN n
   
   """.format()

l_result = my_graph.query_unpaginated(l_query, contextualize=False)
    
print(l_result)



In [ ]:

#  Checking just edges
#

l_query  = """

   MATCH (n) - [r] -> (m)
   RETURN r
   
   """.format()

l_result = my_graph.query_unpaginated(l_query, contextualize=False)
    
print(l_result)


In [ ]:

#  Error,
#
#     
#     Host 0 errors:
#     Traceback (most recent call last):
#       File "/opt/miniconda/lib/python3.8/site-packages/katana_enterprise/worker/worker.py", line 111, in execute
#         value = function(graph)
#       File "/opt/conda/lib/python3.8/site-packages/katana_enterprise/remote/aio/import_data/dataframe_importer/__init__.py", line 512, in remote_import_function
#       File "/opt/miniconda/lib/python3.8/site-packages/katana_enterprise/remote/aio/import_data/dataframe_importer/__init__.py", line 206, in _write_intermediate_parquet_files
#         node_header = "\n".join(_write_dfs_to_parquet(node_df_infos, "nodes")) + "\n"
#       File "/opt/miniconda/lib/python3.8/site-packages/katana_enterprise/remote/aio/import_data/dataframe_importer/__init__.py", line 194, in _write_dfs_to_parquet
#         dask.dataframe.to_parquet(
#       File "/opt/miniconda/lib/python3.8/site-packages/dask/dataframe/io/parquet/core.py", line 963, in to_parquet
#         out = out.compute(**compute_kwargs)
#       File "/opt/miniconda/lib/python3.8/site-packages/dask/base.py", line 315, in compute
#         (result,) = compute(self, traverse=False, **kwargs)
#       File "/opt/miniconda/lib/python3.8/site-packages/dask/base.py", line 598, in compute
#         results = schedule(dsk, keys, **kwargs)
#       File "/opt/miniconda/lib/python3.8/site-packages/distributed/client.py", line 3036, in get
#         results = self.gather(packed, asynchronous=asynchronous, direct=direct)
#       File "/opt/miniconda/lib/python3.8/site-packages/distributed/client.py", line 2210, in gather
#         return self.sync(
#       File "/opt/miniconda/lib/python3.8/site-packages/distributed/utils.py", line 338, in sync
#         return sync(
#       File "/opt/miniconda/lib/python3.8/site-packages/distributed/utils.py", line 405, in sync
#         raise exc.with_traceback(tb)
#       File "/opt/miniconda/lib/python3.8/site-packages/distributed/utils.py", line 378, in f
#         result = yield future
#       File "/opt/miniconda/lib/python3.8/site-packages/tornado/gen.py", line 762, in run
#         value = future.result()
#       File "/opt/miniconda/lib/python3.8/site-packages/distributed/client.py", line 2073, in _gather
#         raise exception.with_traceback(traceback)
#       File "/opt/miniconda/lib/python3.8/site-packages/dask/optimization.py", line 990, in __call__
#         return core.get(self.dsk, self.outkey, dict(zip(self.inkeys, args)))
#       File "/opt/miniconda/lib/python3.8/site-packages/dask/core.py", line 149, in get
#         result = _execute_task(task, cache)
#       File "/opt/miniconda/lib/python3.8/site-packages/dask/core.py", line 119, in _execute_task
#         return func(*(_execute_task(a, cache) for a in args))
#       File "/opt/miniconda/lib/python3.8/site-packages/dask/dataframe/io/parquet/core.py", line 163, in __call__
#         return self.engine.write_partition(
#       File "/opt/miniconda/lib/python3.8/site-packages/dask/dataframe/io/parquet/arrow.py", line 686, in write_partition
#         t = cls._pandas_to_arrow_table(df, preserve_index=preserve_index, schema=schema)
#       File "/opt/miniconda/lib/python3.8/site-packages/dask/dataframe/io/parquet/arrow.py", line 647, in _pandas_to_arrow_table
#         raise ValueError(
#     ValueError: Failed to convert partition to expected pyarrow schema:
#         `ArrowInvalid("Could not convert 'MSH/D009508' with type str: tried to convert to double", 'Conversion failed for column vocabulary_code with type object')`
#     
#     Expected partition schema:
#         id: string
#         LABEL: string
#         vocabulary_code: double
#     
#     Received partition schema:
#         id: string
#         LABEL: string
#         vocabulary_code: string
#     
#     This error *may* be resolved by passing in schema information for
#     the mismatched column(s) using the `schema` keyword in `to_parquet`.
#     
#     ---------------------------------------------------------------------------
#     ValueError                                Traceback (most recent call last)
#     Cell In[3], line 4
#           1 #  Enrich/decode text entry from above
#           2 #
#     ----> 4 l_return, l_pvid = f_enrich(l_text01.value.lower(), my_graph)
#           6 print("Unique Patient Visit Id: %s" % (l_pvid))
#           7 print("")
#     
#     File ~/work/My_KG_NoteBooks/P1_Prospects/13_DataBricks_Google/libraries/SupportFor34.py:422, in f_enrich(i_arg1, i_arg2)
#         420 l_df1, l_df2, l_df3, l_df4, l_df5, l_df6, l_df7 = f_ready_for_graph_int(l_data_asjson, l_uniqid)
#         421    #
#     --> 422 f_insert_into_graph(i_arg2, l_df1, l_df2, l_df3, l_df4, l_df5, l_df6, l_df7)
#         425 return l_data_asjson, l_uniqid
#     
#     File ~/work/My_KG_NoteBooks/P1_Prospects/13_DataBricks_Google/libraries/SupportFor34.py:386, in f_insert_into_graph(i_arg1, i_arg2, i_arg3, i_arg4, i_arg5, i_arg6, i_arg7, i_arg8)
#         369    df_importer.edges_dataframe(
#         370       i_arg7, 
#         371       source_id_space       = "UmlsEntity", 
#        (...)
#         375       type                  = "ALSO_CODED_AS"
#         376       )
#         377    df_importer.edges_dataframe(
#         378       i_arg8, 
#         379       source_id_space       = "UmlsVocabulary", 
#        (...)
#         383       type                  = "ALSO_CODED_AS"
#         384       )
#     --> 386    df_importer.insert()
#         389 return
#     
#     File /opt/conda/lib/python3.8/site-packages/katana_enterprise/async_to_sync.py:259, in AsyncToSync.<locals>.do_wrap.<locals>.wrapper(self, *args, **kwargs)
#         256 @wraps(underlying_func)
#         257 def wrapper(self, *args, **kwargs):
#         258     return registry.async_to_sync(
#     --> 259         underlying_func(
#         260             get_self_func(self),
#         261             *(registry.sync_to_async(a) for a in args),
#         262             **{k: registry.sync_to_async(v) for k, v in kwargs.items()},
#         263         )
#         264     )
#     
#     File /opt/conda/lib/python3.8/site-packages/katana_enterprise/async_to_sync.py:176, in async_to_sync.<locals>.wrapper(timeout, *args, **kwargs)
#         166     registry = AsyncToSyncClassRegistry.get()
#         167     return registry.async_to_sync(
#         168         wait_for(
#         169             async_func(
#        (...)
#         174         )
#         175     )
#     --> 176 return wait_for(async_func(*args, **kwargs), timeout=timeout)
#     
#     File /opt/conda/lib/python3.8/site-packages/katana_enterprise/async_to_sync.py:147, in wait_for(coro, timeout)
#         145 try:
#         146     future = asyncio.run_coroutine_threadsafe(timeout_coro, loop=AsyncRunnerThread.get().loop)
#     --> 147     return future.result()
#         148 except KeyboardInterrupt:
#         149     inner_future.cancel()
#     
#     File /opt/conda/lib/python3.8/concurrent/futures/_base.py:444, in Future.result(self, timeout)
#         442     raise CancelledError()
#         443 elif self._state == FINISHED:
#     --> 444     return self.__get_result()
#         445 else:
#         446     raise TimeoutError()
#     
#     File /opt/conda/lib/python3.8/concurrent/futures/_base.py:389, in Future.__get_result(self)
#         387 if self._exception:
#         388     try:
#     --> 389         raise self._exception
#         390     finally:
#         391         # Break a reference cycle with the exception in self._exception
#         392         self = None
#     
#     File /opt/conda/lib/python3.8/asyncio/tasks.py:455, in wait_for(fut, timeout, loop)
#         450     warnings.warn("The loop argument is deprecated since Python 3.8, "
#         451                   "and scheduled for removal in Python 3.10.",
#         452                   DeprecationWarning, stacklevel=2)
#         454 if timeout is None:
#     --> 455     return await fut
#         457 if timeout <= 0:
#         458     fut = ensure_future(fut, loop=loop)
#     
#     File /opt/conda/lib/python3.8/site-packages/katana_enterprise/remote/aio/import_data/dataframe_importer/__init__.py:539, in DataFrameImporter.__aexit__(self, exc_type, exc_val, exc_tb)
#         537 async def __aexit__(self, exc_type, exc_val, exc_tb):
#         538     if not exc_type:
#     --> 539         await self.execute()
#     
#     File /opt/conda/lib/python3.8/site-packages/katana_enterprise/remote/aio/import_data/dataframe_importer/__init__.py:530, in DataFrameImporter.execute(self)
#         526     csv_options.edge_match_property = edge_global_info.join_key_column or ""
#         528     bulk_ingest_native(csv_options, graph)
#     --> 530 await self.graph.run(remote_import_function)
#     
#     File /opt/conda/lib/python3.8/site-packages/katana_enterprise/remote/aio/graph.py:660, in Graph.run(self, function)
#         658 print(result.stdout, file=sys.stdout, end="")
#         659 print(result.stderr, file=sys.stderr, end="")
#     --> 660 return result.value
#     
#     File /opt/conda/lib/python3.8/site-packages/katana_enterprise/remote/run_result.py:16, in RunResult.value(self)
#          14 @property
#          15 def value(self):
#     ---> 16     self.reraise_if_error()
#          17     assert self.success
#          18     return self._value
#     
#     File /opt/conda/lib/python3.8/site-packages/katana_enterprise/remote/run_result.py:12, in RunResult.reraise_if_error(self)
#          10 def reraise_if_error(self):
#          11     if not self.success:
#     ---> 12         raise self._value
#     
#     File /opt/miniconda/lib/python3.8/site-packages/katana_enterprise/worker/worker.py:153, in execute()
#     
#     File /opt/miniconda/lib/python3.8/site-packages/katana_enterprise/worker/worker.py:111, in execute()
#     
#     File /opt/conda/lib/python3.8/site-packages/katana_enterprise/remote/aio/import_data/dataframe_importer/__init__.py:512, in remote_import_function()
#         510 if MPI.COMM_WORLD.Get_rank() == 0:
#         511     with dask_integration.client():
#     --> 512         node_header, edge_header = parquet_write_fun(graph, node_df_infos, edge_df_infos)
#         514 node_header, edge_header = MPI.COMM_WORLD.bcast((node_header, edge_header), root=0)
#         516 csv_options = CSVImportOptions(node_header, edge_header, stash_uri_for_graph(graph))
#     
#     File /opt/miniconda/lib/python3.8/site-packages/katana_enterprise/remote/aio/import_data/dataframe_importer/__init__.py:206, in _write_intermediate_parquet_files()
#     
#     File /opt/miniconda/lib/python3.8/site-packages/katana_enterprise/remote/aio/import_data/dataframe_importer/__init__.py:194, in _write_dfs_to_parquet()
#     
#     File /opt/miniconda/lib/python3.8/site-packages/dask/dataframe/io/parquet/core.py:963, in to_parquet()
#     
#     File /opt/miniconda/lib/python3.8/site-packages/dask/base.py:315, in compute()
#     
#     File /opt/miniconda/lib/python3.8/site-packages/dask/base.py:598, in compute()
#     
#     File /opt/miniconda/lib/python3.8/site-packages/distributed/client.py:3036, in get()
#     
#     File /opt/miniconda/lib/python3.8/site-packages/distributed/client.py:2210, in gather()
#     
#     File /opt/miniconda/lib/python3.8/site-packages/distributed/utils.py:338, in sync()
#     
#     File /opt/miniconda/lib/python3.8/site-packages/distributed/utils.py:405, in sync()
#     
#     File /opt/miniconda/lib/python3.8/site-packages/distributed/utils.py:378, in f()
#     
#     File /opt/miniconda/lib/python3.8/site-packages/tornado/gen.py:762, in run()
#     
#     File /opt/miniconda/lib/python3.8/site-packages/distributed/client.py:2073, in _gather()
#     
#     File /opt/miniconda/lib/python3.8/site-packages/dask/optimization.py:990, in __call__()
#     
#     File /opt/miniconda/lib/python3.8/site-packages/dask/core.py:149, in get()
#     
#     File /opt/miniconda/lib/python3.8/site-packages/dask/core.py:119, in _execute_task()
#     
#     File /opt/miniconda/lib/python3.8/site-packages/dask/dataframe/io/parquet/core.py:163, in __call__()
#     
#     File /opt/miniconda/lib/python3.8/site-packages/dask/dataframe/io/parquet/arrow.py:686, in write_partition()
#     
#     File /opt/miniconda/lib/python3.8/site-packages/dask/dataframe/io/parquet/arrow.py:647, in _pandas_to_arrow_table()
#     
#     ValueError: Failed to convert partition to expected pyarrow schema:
#         `ArrowInvalid("Could not convert 'MSH/D009508' with type str: tried to convert to double", 'Conversion failed for column vocabulary_code with type object')`
#     
#     Expected partition schema:
#         id: string
#         LABEL: string
#         vocabulary_code: double
#     
#     Received partition schema:
#         id: string
#         LABEL: string
#         vocabulary_code: string
#     
#     This error *may* be resolved by passing in schema information for
#     the mismatched column(s) using the `schema` keyword in `to_parquet`.

